# LVQ. <br>Really is a new thinking. 

I saw how it works and how it is calculated. There is **really**, **really** simple implementation to which you will say "Wow this simple?" But I suppose that in the *sklearn* or in the *actual implementation* there would be *slightly more* to it like real neural nets.

The way we are going to follow is **without neural net**, because you will see that we don't actully need any kind of that structure. We just want to update the values in each *layer*. <br>
—<br>
So, let's go ∞

# 

In [1]:
# Great stuff to import!
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# The same data that we used for KNN
X1 = [3.393533211, 3.110073483, 1.343808831,
      3.582294042, 2.280362439, 7.423436942,
      5.745051997, 9.172168622, 7.792783481,
      7.939820817]

X2 = [2.331273381, 1.781539638, 3.368360954,
      4.67917911, 2.866990263, 4.696522875,
      3.533989803, 2.511101045, 3.424088941,
      0.791637231]

y = [0] * 5 + [1] * 5

In [3]:
df = pd.DataFrame({"x1":X1, 
                   "x2":X2,
                   "y":y})
df

,x1,x2,y
0,3.393533,2.331273,0
1,3.110073,1.781540,0
2,1.343809,3.368361,0
3,3.582294,4.679179,0
4,2.280362,2.866990,0
5,7.423437,4.696523,1
6,5.745052,3.533990,1
7,9.172169,2.511101,1
8,7.792783,3.424089,1
9,7.939821,0.791637,1


# 

Now, that we have the data - we have some **basic steps**:
1. Set how many **layers**, **initial learning rate**, and **epochs**.
2. **Initialize** the *so called* neural net with some random values from training data and also the y.
3. For that epoch and that learning rate, **take one by one** row from training dataset.
4. For that row, **compare the distances** between all layers in the so called NN.
5. Pick the **least distanced** data point. Yes just 1.
6. We call it **BMU** (Best Measurable Unit).
7. **If** the BMU's class (from NN) is **same as** that row's class, then ***promote*** that BMU's features based on the learning rate currently is.
8. **If** the BMU's class is **not same as** that row's class, then ***demote*** that BMU's features based on the learning rate currently is.
9. Then **fetch next row** and repeat steps 4-8 again till the entire dataset is traversed one time.
10. After 9th step, we will have **a dataset traversed**. It is an **epoch**. Update the learning rate based on the formula given above.
11. Run step 4-10 for **n number of epochs** set already.
12. After 11th step (traversing for n number of times) we will have our learnt **NN ready**. With the updated attributes.

*(Actully I don't like to call the final representation as NN, because NN is an entirely different thing. But for the sake of this algorithm, we are gonna call is as such)*.

# 

In [4]:
# 1. Setting layers, LR, epochs
n_layers = 4
LearningRate = 0.7
epochs = 10

In [22]:
# Kindly ignore this code
def styling_1(df):
    purple = "#a66efa"
    blue = "#4287f5"
    pink = "#f54295"
    green = "#42e0f5"
    
    df_styler = pd.DataFrame('', index= df.index, columns= df.columns)
    df_styler.iloc[3, 0] = f'background-color: {pink}'
    df_styler.iloc[9, 1] = f'background-color: {pink}'
    
    df_styler.iloc[8, 0] = f'background-color: {blue}'
    df_styler.iloc[0, 1] = f'background-color: {blue}'
    
    df_styler.iloc[9, 0] = f'background-color: {green}'
    df_styler.iloc[4, 1] = f'background-color: {green}'
    
    df_styler.iloc[0, 0] = f'background-color: {purple}'
    df_styler.iloc[3, 1] = f'background-color: {purple}'
    return df_styler

df.style.apply(styling_1, axis= None)

,x1,x2,y
0,3.393533,2.331273,0
1,3.110073,1.781540,0
2,1.343809,3.368361,0
3,3.582294,4.679179,0
4,2.280362,2.866990,0
5,7.423437,4.696523,1
6,5.745052,3.533990,1
7,9.172169,2.511101,1
8,7.792783,3.424089,1
9,7.939821,0.791637,1


In [89]:
# 2. Initialize the NN 
# (we will pick the numbers randomly but for now we will
#  pick from the book to tallyup the results)
NN = pd.DataFrame({"x1": [3.582294, 7.792783, 7.939821, 3.393533],
                   "x2": [0.791637, 2.331273, 2.866990, 4.679179],
                   "y" : [0       , 0       , 1       , 1       ]})

In [11]:
# Kindly ignore this code
def styling_2(df):
    df_styler = pd.DataFrame('', index= df.index, columns= df.columns)
    df_styler.iloc[0, :-1] = 'background-color: #f54295'
    df_styler.iloc[1, :-1] = 'background-color: #4287f5'
    df_styler.iloc[2, :-1] = 'background-color: #42e0f5'
    df_styler.iloc[3, :-1] = 'background-color: #a66efa'
    return df_styler

NN.style.apply(styling_2, axis= None)

,x1,x2,y
0,3.582294,0.791637,0
1,7.792783,2.331273,0
2,7.939821,2.866990,1
3,3.393533,4.679179,1


In [30]:
# 3. Take one row at a time and update
### taking only first row ↓
row = df.iloc[0].values[:-1]

# 4. Calculate the distance for all rows in NN
def euclidean_distance(A, B):
    return ((A - B) ** 2).sum(1) ** 0.5

### passing whole NN as `B` ↓
distances = euclidean_distance(row, NN.iloc[:, :-1].values)
distances

array([1.55116434, 4.39924979, 4.57774234, 2.34790562])

↑ This shows that very **first layer** in the NN with `1.55116` euclidean distance - is the candidate to be our BMU. 

In [46]:
# 5. Pick the least distanced layer and call it BMU
# 6. And call it BMU
bmu = NN.iloc[distances.argmin()]
bmu

x1    3.582294
x2    0.791637
y     0.000000
Name: 0, dtype: float64

`.argmin` will return the index of the least distanced row. Here, we have it on `0th` index so now the updation will happen on that row.

In [41]:
# 7. or 8. If class same then promote otherwise demote
bmu["y"] == df.iloc[0]["y"]

True

It does! So we will promote the BMU with the respective values.

## $ \text{layer} = \text{layer} + \text{LearningRate} \times (x - \text{layer}) $
###### Here, $\text{layer} = \text{bmu}$

In [42]:
new_bmu = bmu.iloc[:-1] + LearningRate * (row - bmu.iloc[:-1])
new_bmu

x1    3.450161
x2    1.869382
Name: 0, dtype: float64

In [47]:
# Putting that in NN
NN.iloc[0, :-1] = new_bmu

In [49]:
def styling(df):
    df_styler = pd.DataFrame('', index= df.index, columns= df.columns)
    df_styler.iloc[0, :-1] = 'background-color: #f54295'
    return df_styler

NN.style.apply(styling, axis= None)

,x1,x2,y
0,3.450161,1.869382,0
1,7.792783,2.331273,0
2,7.939821,2.866990,1
3,3.393533,4.679179,1


The BMU (in pink color) is now updated with new values. 


—<br>
Now, as out pseudo code goes - we have to iterate through whole dataset for `n` times and update the values as required. So, let's do that.

In [90]:
for row in df.values:
    distances = euclidean_distance(row[:-1], NN.iloc[:, :-1].values)
    bmu_at = distances.argmin()
    bmu = NN.iloc[bmu_at]
    if bmu["y"] == row[-1]:
        new_bmu = bmu.iloc[:-1] + LearningRate * (row[:-1] - bmu.iloc[:-1])
    else:
        new_bmu = bmu.iloc[:-1] - LearningRate * (row[:-1] - bmu.iloc[:-1])
    NN.iloc[bmu_at, :-1] = new_bmu

In [91]:
NN

,x1,x2,y
0,2.559884,2.549261,0
1,6.048388,3.195023,0
2,7.343461,3.512290,1
3,5.700572,6.239052,1


Now, after an epoch (only 1) we have this ↑ NN values. Now, of course we can iterate over it more and more... but for now let's go an predict. 

*(I know the part of "Updating Learning Rate" is still not implemented here... but it will be included when we make the class for this LVQ).*

## Prediction<br>—

Now, you can say that, we are using KNN here. Yes. For prediction we have to take the new instances and for defined `k` value we have to ask for votes. 

The main advantage is that, our dataset now has shrunk down from 10 rows to just 4. Imagine a dataset contain 10,000 rows and we shrunk it down to 100 rows only! Much faster algorithm, isn't it?

In [174]:
# Our own KNN implementation is being imported
from KNN import KNN

In [98]:
X = NN.drop("y", axis=1)
y = NN["y"]

In [99]:
model = KNN(X, y)

In [106]:
df["pred"] = model.predict(df[["x1", "x2"]], k=2)
df

,x1,x2,y,pred
0,3.393533,2.331273,0,0
1,3.110073,1.781540,0,0
2,1.343809,3.368361,0,0
3,3.582294,4.679179,0,0
4,2.280362,2.866990,0,0
5,7.423437,4.696523,1,1
6,5.745052,3.533990,1,0
7,9.172169,2.511101,1,1
8,7.792783,3.424089,1,1
9,7.939821,0.791637,1,1


In [108]:
(df.y == df.pred).sum() / df.shape[0]

0.9

It is getting `90%` accuracy. Of course some more epochs are needed.

# 

## Making the compact class<br>— 
*Here again, I will try to implement the same in numpy for much faster work*

In [285]:
# %load KNN.py

import statistics
import numpy as np

class KNN:
    """
    This model is so great. It also can work on 4 different
    types of distance metrics.
    
    First time, I tried to make a model entirely in numpy.
    Haven't used a single bit of pandas. It makes numpy stronger
    and calculation faster.
    
    
    How To
    ------
    
    >>> model = KNN(X, y)
    >>> pred = model.predict(X, k=3, dis_type="euclidean")
    """
    def __init__(self, X: np.ndarray, y: list):
        X = np.array(X)
        y = np.array(y)
        if X.ndim != 2:
            raise NotImplementedError(\
            """
            The dimention of the features 
            must be 2D.
            """)
        if (len(X) != len(y)) or (y.ndim != 1):
            raise NotImplementedError(\
            """
            The length of features 
            and target mismatched.
            """)
        self.stored_X = X
        self.stored_y = y
        
    
    def predict(self, X: np.ndarray, k: int, dis_type="euclidean", p=None):
        X = np.array(X)
        self.dis_type = dis_type
        if (self.dis_type == "minkowaski") and (p == None):
            raise NotImplementedError("Please provide `p` value.")
        self.p = p    
        if X.ndim != 2:
            raise NotImplementedError(\
            """
            The dimention of the features 
            must be 2D.
            """)
        pred_classes = []
        for each_row in X:
            distance = self.get_distance(row=each_row)
            sorted_k_indexes = np.argsort(distance)[:k]
            pred_class = statistics.mode(self.stored_y[sorted_k_indexes])
            pred_classes.append(pred_class)
        return pred_classes
    
    def get_distance(self, row):
        if self.dis_type == "euclidean":
            return ((row - self.stored_X) ** 2).sum(1) ** 0.5
        elif self.dis_type == "manhattan":
            return abs((row - self.stored_X)).sum(1)
        elif self.dis_type == "hamming":
            return abs(row - self.stored_X).sum(1) / len(row)
        elif self.dis_type == "minkowaski":
            return (abs(row - self.stored_X) ** self.p).sum(1) ** (1 / self.p)
        else:
            raise NotImplementedError(\
            f"""
            The distance type chosen is `{self.dis_type}`.
            Please choose from: 
            • euclidean
            • manhattan
            • hamming
            • minkowaski
            """)


In [286]:
%%writefile LVQ_KNN.py

import math
import numpy as np
from KNN import KNN


class LVQ_KNN:
    """
    This class is implemented to build LVQ model.
    It gives a nice range of hyperparameters to tune
    and works much faster than the traditional KNN.
    
    For the prediction part, it uses KNN though, but
    with the less number of layers, it works amazingly fast.
    
    How To
    ------
        # Prepare X and y
    >>> X = iris.drop("species", axis=1)
    ... y = iris["species"]
    
        # Train
    >>> model = LVQ_KNN(X, y, n_layers=10, 
                        learning_rate=0.3, epochs=200,
                        learning_mode="static / dynamic")
    >>> model.predict(X)                              
    """
    def __init__(self, X, y, n_layers, learning_rate=0.3, epochs=2,
                 learning_mode="dynamic"):
        self.X = np.array(X); self.y = np.array(y)
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.learning_mode = learning_mode
        
        if X.ndim != 2:
            raise NotImplementedError(\
            """
            The dimention of the features 
            must be 2D.
            """)
        if (len(X) != len(y)) or (y.ndim != 1):
            raise NotImplementedError(\
            """
            The length of features 
            and target mismatched.
            """)
        if n_layers > len(self.X):
            raise NotImplementedError(\
            """
            `n_layers` must be less than
            total rows in dataset.
            """)
            
        self.NN = self.randomize_init()
        for epoch in range(self.epochs):
            # LearningRate = LearningRate × (1 − 𝑛𝑡ℎEpoch / TotalEpochs)
            if self.learning_mode == "dynamic":
                self.learning_rate = self.learning_rate * (1 - epoch / self.epochs)
            self.compress_data()
        
        
    def randomize_init(self):
        shuffled_X = None
        for column in range(self.X.shape[1]):
            mask = np.random.permutation(self.X.shape[0])[:self.n_layers]
            if shuffled_X is None:
                shuffled_X = self.X[mask, column]
            else:
                shuffled_X = np.c_[shuffled_X, self.X[mask, column]]
        self.stratified_y_column = self.stratified_sampling()[: self.n_layers]
        return shuffled_X
     
        
    def compress_data(self):
        for row in self.X:
            distances = self.euclidean_distance(row, self.NN)
            bmu_at = distances.argmin()
            bmu = self.NN[bmu_at]
            if self.y[bmu_at] == self.stratified_y_column[bmu_at]:
                new_bmu = bmu + self.learning_rate * (row - bmu)
            else:
                new_bmu = bmu - self.learning_rate * (row - bmu)
            self.NN[bmu_at] = new_bmu
    
    
    def predict(self, X, k):
        model = KNN(self.NN, self.stratified_y_column)
        return model.predict(X, k=k)
    
    
    @staticmethod
    def euclidean_distance(A, B):
        return ((A - B) ** 2).sum(1) ** 0.5
    
    
    def stratified_sampling(self):
        total = len(self.y)
        new_y = []
        for class_, count in zip(*np.unique(self.y, return_counts=True)):
            new_count = math.ceil((count * self.n_layers) / total)
            new_y.extend(np.full(new_count, class_))
        return new_y

Writing LVQ_KNN.py


In [224]:
X = df[["x1", "x2"]]
y = df["y"]

In [244]:
model = LVQ_KNN(X, y, 4, 0.001, epochs=200, learning_mode="static")

In [245]:
model.predict(X, k=3)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

The randomize nature sometimes gives in appropriate results... but for now it seems to be working.

Let's try it on the new dataset, our old `iris`.

In [246]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [269]:
X = iris.drop("species", axis=1)
y = iris["species"]

model = LVQ_KNN(X, y, n_layers=20,
               learning_rate=0.01, epochs=500, 
               learning_mode="static")

In [283]:
pred = model.predict(X, k=12)
(pred == y).sum() / len(y)

0.66

Not a good accuracy at all... but still it is there!

# 

# That's it!
Next up, we will learn a new algorithm!